In [1]:
#######################################
# Auto-reload development imports
#######################################

%load_ext autoreload
%autoreload 2

In [2]:
#######################################
#imports
#######################################

import os, sys
dir0 = os.path.split(os.getcwd())[0]
if dir0 not in sys.path:
    sys.path.append(dir0)
import mlproc_dev as mlp

In [3]:
#######################################
# LARI login
#######################################

#lari_id='fdb573a66f50' ## lari_ccblin008
#lari_id='19b775bb238f' ## lari_g5ub
#lari_id='133898b2b079' ## lari_jm
lari_id='cb48a51bf9e5'
#lari_id=''

mlp.lariLogin(lari_id,lari_passcode=None)

Enter processing passcode for lari node cb48a51bf9e5


In [4]:
#######################################
# Import sorting pipelines
#######################################

from mountainsort4_v1_dev import sort_dataset as ms4_sort_dataset
from summarize_sorting_results_dev import summarize_sorting_results

In [5]:
#######################################
# Create output directory
#######################################

output_base_dir=os.getcwd()+'/output'
#import shutil
#if os.path.exists(output_base_dir):
#    shutil.rmtree(output_base_dir)
if not os.path.exists(output_base_dir):
    print('Creating directory: {}'.format(output_base_dir))
    os.mkdir(output_base_dir)
else:
    print('Directory already exists: {}'.format(output_base_dir))

Directory already exists: /home/magland/src/mountainsort_examples/spikeforest/output


In [6]:
#######################################
# Set up datasets
#######################################

datasets=[]
datasets.append({
    "id":"synth_tetrode_30min",
    "raw_path":"kbucket://b5ecdf1474c5/datasets/synth_datasets/datasets/synth_tetrode_30min"
});
datasets.append({
    "id":"synth_tetrode_120min",
    "raw_path":"kbucket://b5ecdf1474c5/datasets/synth_datasets/datasets/synth_tetrode_120min"
});
#datasets.append({
#    "id":"synth_16ch_30min",
#    "raw_path":"kbucket://b5ecdf1474c5/datasets/synth_datasets/datasets/synth_16ch_30min"
#});

#datasets=[datasets[0]]

print('Number of datasets: {}'.format(len(datasets)))

Number of datasets: 2


In [7]:
#######################################
# Set up algorithms
#######################################

verbose=''

def ms4_thr3(dataset_dir,output_dir):
    ms4_sort_dataset(
        dataset_dir=dataset_dir,
        output_dir=output_dir,
        adjacency_radius=-1,
        detect_threshold=3,
        opts={'verbose':verbose}
    )
    
def ms4_thr4(dataset_dir,output_dir):
    ms4_sort_dataset(
        dataset_dir=dataset_dir,
        output_dir=output_dir,
        adjacency_radius=-1,
        detect_threshold=4,
        opts={'verbose':verbose}
    )

algs=[]
algs.append({
    "name":"ms4_thr3",
    "run":ms4_thr3
})
algs.append({
    "name":"ms4_thr4",
    "run":ms4_thr4
})

print('Number of algorithms: {}'.format(len(algs)))

Number of algorithms: 2


In [8]:
#######################################
# Collect runs (algorithm/dataset pairs)
#######################################

runs=[]
for i_ds in range(len(datasets)):
    for i_alg in range(len(algs)):
        runs.append({"dataset":datasets[i_ds],"alg":algs[i_alg]})
runs=[runs[0]]

print('Number of runs: {}'.format(len(runs)))

Number of runs: 1


In [9]:
#######################################
# Define the pipeline for the run
#######################################

def handle_run(run):
    DS=run['dataset']
    ALG=run['alg']
    print(':::: Applying '+ALG['name']+' to '+DS['id'])
    dsdir=DS['raw_path']
    dsid=DS['id']
    algname=ALG['name']
    output_dir=output_base_dir+'/'+dsid+'--'+algname
    ALG['run'](
        dataset_dir=dsdir,
        output_dir=output_dir
    )
    #summarize_sorting_results(
    #    sorting_output_dir=output_dir,
    #    output_dir=output_dir+'/summary',
    #    opts={'verbose':verbose}
    #)

In [14]:
#######################################
# RUN THE PIPELINE
#######################################

mlp.initPipeline()
for run in runs:
    handle_run(run)
mlp.runPipeline()

:::: Applying ms4_thr3 to synth_tetrode_30min


Output()

Output()

Output()

Output()

Output()

Output()

Output()

Output()

Running pipeline...
Running: ml-run-process ephys.bandpass_filter -i timeseries:kbucket://b5ecdf1474c5/datasets/synth_datasets/datasets/synth_tetrode_30min/raw.mda -o timeseries_out:/home/magland/src/mountainsort_examples/spikeforest/output/synth_tetrode_30min--ms4_thr3/filt.mda.prv -p freq_min:300 samplerate:30000 freq_max:6000 --lari_out=tmp.ephys.bandpass_filter.lari_out.FKqJK8iPR3.json --verbose=
JOBS: pending:7 running:1 finished:0
Running: ml-run-process ephys.whiten -i timeseries:/home/magland/src/mountainsort_examples/spikeforest/output/synth_tetrode_30min--ms4_thr3/filt.mda.prv -o timeseries_out:/home/magland/src/mountainsort_examples/spikeforest/output/synth_tetrode_30min--ms4_thr3/pre.mda.prv -p  --lari_out=tmp.ephys.whiten.lari_out.ydL28RwhUE.json --verbose=
JOBS: pending:6 running:1 finished:1
Running: ml-run-process ms4alg.sort -i geom:kbucket://b5ecdf1474c5/datasets/synth_datasets/datasets/synth_tetrode_30min/geom.csv timeseries:/home/magland/src/mountainsort_examples/sp

In [15]:
import vdom

def show_result(run):
    DS=run['dataset']
    ALG=run['alg']
    run_title=ALG['name']+' : '+DS['id']
    dsdir=DS['raw_path']
    dsid=DS['id']
    algname=ALG['name']
    output_dir=output_base_dir+'/'+dsid+'--'+algname
    
    display(vdom.h3(run_title))
    
    url='https://ephys-viz.herokuapp.com/?view=dataset&dataset={}'.format(dsdir)
    display(vdom.a('Dataset',href=url,target='_blank'))
    
    url='https://ephys-viz.herokuapp.com/?view=dataset&dataset={}&firings={}'.format(dsdir,'test')
    display(vdom.a('Sorting result',href=url,target='_blank'))
    
for run in runs:
    show_result(run)

<h3 />

<a />

<a />